In [26]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import cm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import dotenv_values
import random

In [27]:
random.seed(42) 

In [28]:
config = dotenv_values('.env')

# Tracks

In [66]:
def fix_tracks(tracks: list, printLong: bool = True) -> list:
    """
    Reemplaza nombres de canciones. Si no está en el diccionario, se mantiene tal cual.
    """

    replacements = {
        "Igual Que Un Ángel (with Peso Pluma)": "Igual Que Un Ángel",
        "Rocket Man (I Think It's Going To Be A Long, Long Time)": "Rocket Man",
        "All Night Long (All Night)": "All Night Long",
        "Un'Estate Italiana (Notti Magiche)": "Un'Estate Italiana",
        "Praise You (Radio Edit)": "Praise You",
        "Vente pa' ca (feat. maluma)": "Vente pa' ca",
        "Dime que me quieres (bring a little lovin)": "Dime que me quieres",
        "La temperatura (feat. eli palacios)": "La temperatura",
    }
    
    tracks = [item["track"]["name"].split(" -")[0] for item in tracks if item["track"]]
    tracks = [replacements.get(t, t) for t in tracks]
    # sacamos cosas como " - Remix" o " - Edit"
    tracks = [t.capitalize() for t in tracks]

    if printLong:
        for track in range(len(tracks)):
            if len(tracks[track]) > 25:
                print(f"FYI: este track quizá es demasiado largo: {tracks[track]}")
    
    return tracks

def get_tracks(url: str) -> list:
    """
    Obtiene los tracks de una playlist de Spotify.
    """
    
    try:
        auth_manager = SpotifyClientCredentials(
            client_id=config["SPOTIFY_CLIENT_ID"],
            client_secret=config["SPOTIFY_CLIENT_SECRET"],
        )
        sp = spotipy.Spotify(auth_manager=auth_manager)

        results = sp.playlist_items(url)
        tracks = results["items"]

        # Spotify devuelve resultados paginados (de a 100), nos aseguramos de traerlos todos
        while results["next"]:
            results = sp.next(results)
            tracks.extend(results["items"])


        tracks = fix_tracks(tracks)
        print(f"Se encontraron {len(tracks)=} tracks")
        return tracks
    except Exception as e:
        print(f"Error al conectar con Spotify: {e}")
        return None

In [67]:
tracks = get_tracks(config["SPOTIFY_PLAYLIST_URL"])
with open("tracks.txt", "w") as f:
    for track in tracks:
        f.write(f"{track}\n")

FYI: este track quizá es demasiado largo: Vente pa' ca (feat. maluma)
FYI: este track quizá es demasiado largo: Seguir viviendo sin tu amor
FYI: este track quizá es demasiado largo: La temperatura (feat. eli palacios)
FYI: este track quizá es demasiado largo: Dime que me quieres (bring a little lovin)
Se encontraron len(tracks)=72 tracks


# Cartones

In [70]:
NUM_CARTONES = 20
NUM_TRACKS_POR_CARTON = 12

In [71]:
def sampleTracks(tracks, sampleSize):
    """
    Genera un carton con tracks aleatorios
    """
    return random.sample(tracks, sampleSize)

In [72]:
def generar_cartones(
    tracks: list, 
    numCartones: int = NUM_CARTONES, 
    tracksPorCarton: int = NUM_TRACKS_POR_CARTON
) -> dict:
    """
    Genera M cartones con N tracks aleatorios
    """
    if len(tracks) < tracksPorCarton:
        raise ValueError(
            f"No hay suficientes tracks ni para un cartón (hay {len(tracks)} disponibles, se necesitan {tracksPorCarton})"
        )

    cartones = dict()
    for i in range(1, numCartones + 1):
        cartones[i] = sampleTracks(tracks, sampleSize=tracksPorCarton)
        # Evitamos cartones iguales a otros
        if i > 1:
            for j in range(1, i):
                while len(set(cartones[i]) & set(cartones[j])) == tracksPorCarton:
                    cartones[i] = sampleTracks(tracks, sampleSize=tracksPorCarton)
    return cartones


In [73]:
cartones = generar_cartones(tracks, numCartones=NUM_CARTONES, tracksPorCarton=NUM_TRACKS_POR_CARTON)

# PDF

In [75]:
fontName = "Chalkboard"
pdfmetrics.registerFont(TTFont(fontName, f'{fontName}.ttc'))

In [76]:
def crear_pdf_cartones(
    cartones: dict, 
    filename: str = "bingo_musical.pdf", 
    cartones_por_pagina=2,
    filas=4,
    columnas=3
):
    """
    Crea un PDF con los cartones para imprimir
    Grilla parametrizable (default 4x3 = 12 tracks por cartón)
    """
    doc = SimpleDocTemplate(
        filename,
        pagesize=A4,
        topMargin=1.2 * cm,
        bottomMargin=1.2 * cm,
        leftMargin=1 * cm,
        rightMargin=1 * cm,
    )

    elementos = []
    styles = getSampleStyleSheet()

    # Estilo personalizado para el título
    titulo_style = ParagraphStyle(
        "TituloCarton",
        parent=styles["Heading2"],
        alignment=1,  # Centrado
        fontSize=20,
        spaceAfter=0.6 * cm,
        textColor=colors.black,
        fontName="Helvetica-Bold",
        # borderWidth=1,
        # borderColor=colors.black,
        # borderPadding=8,
        # backColor=colors.white,
    )

    # Estilo para las canciones
    cancion_style = ParagraphStyle(
        "Cancion",
        parent=styles["Normal"],
        fontSize=10,
        alignment=1,  # Centrado
        leading=11,
        wordWrap="CJK",  # Permite mejor wrap de texto
        fontName=fontName,
        textColor=colors.black,
    )

    # Procesar cada cartón
    cartones_procesados = 0
    for numero_carton, tracks in cartones.items():
        # Título del cartón
        titulo = Paragraph(f"CARTÓN #{numero_carton}", titulo_style)
        elementos.append(titulo)

        # Organizar tracks en matriz filas x columnas
        tabla_data = []
        for fila in range(filas):
            fila_data = []
            for col in range(columnas):
                idx = fila * columnas + col
                if idx < len(tracks):
                    track = tracks[idx]
                    # No acortamos los nombres, los dejamos completos
                    fila_data.append(Paragraph(track, cancion_style))
                else:
                    fila_data.append("")
            tabla_data.append(fila_data)

        # Calcular ancho de columna para que ocupe todo el ancho disponible
        ancho_disponible = A4[0] - (2 * cm) # Ancho página - márgenes izq/der
        ancho_columna = ancho_disponible / columnas

        # Crear tabla con dimensiones apropiadas
        tabla = Table(
            tabla_data, 
            colWidths=[ancho_columna] * columnas, 
            rowHeights=[2.2 * cm] * filas
        )
        
        tabla.setStyle(
            TableStyle(
                [
                    # Fondo blanco para todas las celdas
                    ("BACKGROUND", (0, 0), (-1, -1), colors.white),
                    ("TEXTCOLOR", (0, 0), (-1, -1), colors.black),
                    ("ALIGN", (0, 0), (-1, -1), "CENTER"),
                    ("VALIGN", (0, 0), (-1, -1), "MIDDLE"),
                    ("FONTNAME", (0, 0), (-1, -1), "Helvetica"),
                    ("FONTSIZE", (0, 0), (-1, -1), 9),
                    # Bordes externos más gruesos para el marco
                    (
                        "LINEBELOW",
                        (0, -1), # Última fila
                        (-1, -1),
                        2,
                        colors.black,
                    ),  # Borde inferior grueso
                    (
                        "LINEABOVE",
                        (0, 0),
                        (-1, 0),
                        2,
                        colors.black,
                    ),  # Borde superior grueso
                    (
                        "LINEBEFORE",
                        (0, 0),
                        (0, -1),
                        2,
                        colors.black,
                    ),  # Borde izquierdo grueso
                    (
                        "LINEAFTER",
                        (-1, 0),
                        (-1, -1),
                        2,
                        colors.black,
                    ),  # Borde derecho grueso
                    # Bordes internos más delgados y elegantes
                    ("INNERGRID", (0, 0), (-1, -1), 0.5, colors.grey),
                    # Padding generoso para que se vea espacioso
                    ("LEFTPADDING", (0, 0), (-1, -1), 8),
                    ("RIGHTPADDING", (0, 0), (-1, -1), 8),
                    ("TOPPADDING", (0, 0), (-1, -1), 6),
                    ("BOTTOMPADDING", (0, 0), (-1, -1), 6),
                    # Sombra sutil en los bordes (simulada con bordes adicionales)
                    ("BOX", (0, 0), (-1, -1), 2, colors.black),
                ]
            )
        )

        elementos.append(tabla)
        cartones_procesados += 1

        # Agregar espacio entre cartones o salto de página
        if cartones_procesados % cartones_por_pagina == 0 and cartones_procesados < len(
            cartones
        ):
            elementos.append(PageBreak())
        else:
            elementos.append(Spacer(1, 0.8 * cm))

    # Generar PDF
    doc.build(elementos)
    print(f"PDF generado: {filename}")


In [77]:
crear_pdf_cartones(cartones, "bingo_cumple.pdf")

PDF generado: bingo_cumple.pdf


In [78]:
tracksUsados = set()
for _, t in cartones.items():
    tracksUsados.update(t)

In [80]:
if set(tracks) == tracksUsados:
    print("Se usaron todos los tracks de la playlist.")
else:
    print("No se usaron todos los tracks de la playlist.")
    tracks_no_usados = set(tracks) - set(tracksUsados)
    print(f"Tracks no usados: {tracks_no_usados}")

No se usaron todos los tracks de la playlist.
Tracks no usados: {'Wrecking ball', 'Amores como el nuestro', 'La rubia del avión'}
